In [12]:
!pip install pandas -q
!pip install denseweight

import numpy as np
import pandas as pd
from denseweight import DenseWeight


# File paths
train_file = "./extra_lin_around_mean_train_low_std_range_very_low_std_standardized_log1p.csv"
test_file = "./extra_lin_around_mean_test_low_std_range_very_low_std_standardized_log1p.csv"

# Load the datasets
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

dw = DenseWeight(alpha=1.0)
print(np.random.normal(size=1000).shape)
print(pd.concat([train_df["Label"], test_df["Label"]]).to_numpy().shape)
weights = dw.fit(pd.concat([train_df["Label"], test_df["Label"]]).to_numpy())
train_weights = dw(train_df["Label"].to_numpy())
test_weights = dw(test_df["Label"].to_numpy())
print(dw([6]))

# Display the first few rows of each dataset
print("Train Data:")
print(train_df.info())

print("\nTest Data:")
print(test_df.info())



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


(1000,)
(1000000,)
[2.09604394]
Train Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  800000 non-null  float64
 1   Series2  800000 non-null  float64
 2   Series3  800000 non-null  float64
 3   Label    800000 non-null  float64
dtypes: float64(4)
memory usage: 24.4 MB
None

Test Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  200000 non-null  float64
 1   Series2  200000 non-null  float64
 2   Series3  200000 non-null  float64
 3   Label    200000 non-null  float64
dtypes: float64(4)
memory usage: 6.1 MB
None


In [13]:
train_df.describe()

,Series1,Series2,Series3,Label
count,800000.000000,800000.000000,800000.000000,8.000000e+05
mean,-0.000727,0.000195,0.000584,3.165487e-01
std,0.999889,0.999988,0.999805,5.774442e-01
min,-2.311488,-1.714816,-1.714816,0.000000e+00
25%,-0.457694,-0.848747,-0.848747,4.289651e-09
50%,-0.000698,-0.017321,0.017321,9.727914e-02
75%,0.456647,0.883390,0.883390,4.120072e-01
max,2.311415,1.714816,1.714816,5.991320e+00


In [14]:
# install norse and pytorch
!pip install sinabs


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch

class DataFrameDataset(Dataset):
    def __init__(self, dataframe):
        self.inputval = dataframe["Series1"]
        self.mean = dataframe["Series2"]
        self.std = dataframe["Series3"]
        self.labels = dataframe["Label"]

    def __len__(self):
        return len(self.inputval)

    def __getitem__(self, idx):
        inputval = self.inputval.iloc[idx]
        mean = self.mean.iloc[idx]
        std = self.std.iloc[idx]
        labels = self.labels.iloc[idx]
        return inputval, mean, std, labels
    
# Create the datasets
train_dataset = DataFrameDataset(train_df)
test_dataset = DataFrameDataset(test_df)


In [16]:
from torch.utils.data import WeightedRandomSampler

train_sampler = WeightedRandomSampler(weights=train_weights, num_samples=len(train_weights))


In [17]:
# Encode input data
train_loader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_size=64) #, shuffle=True)


In [18]:
import torch.nn as nn
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [19]:
import torch

TRAINING_PREFIX = "dense_weights_extra_lin"

ann = nn.Sequential(
    nn.Linear(3, 256),  # Input layer: 3 features (mu, sigma, x)
    nn.ReLU(),
    #nn.Dropout(0.2),
    nn.Linear(256, 256),  # first hidden layer
    nn.ReLU(),
    #nn.Dropout(0.2),
    nn.Linear(256, 256),  # second hidden layer
    nn.ReLU(),
    #nn.Dropout(0.2),
    nn.Linear(256, 1)    # Output layer: single value for f(x; mu, sigma)
)

# Load checkpoint
#ann.load_state_dict(torch.load(f"results/{TRAINING_PREFIX}/ann_epoch_300.pth"))

ann.to(device)

total_params = sum(p.numel() for p in ann.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params}")


Total Parameters: 132865


In [20]:
from sinabs.from_torch import from_model

num_time_steps_per_sample = 100

sinabs_model = from_model(ann, input_shape=(3,), add_spiking_output=True, synops=False, num_timesteps=num_time_steps_per_sample)
sinabs_model

Network(
  (spiking_model): Sequential(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (6): Linear(in_features=256, out_features=1, bias=True)
    (spike_output): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
  )
  (analog_model): Sequential(
    (0): Linear(in_features=3, out_features

In [21]:
!pip install torcheval -q

from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
from torcheval.metrics import R2Score


def create_loss_plot():
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Train Loss', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Eval Loss', line=dict(color='orange')))

    # Configure layout
    fig.update_layout(title='Training and Evaluation Losses',
                    xaxis_title='Epoch',
                    yaxis_title='Loss',
                    template='plotly_dark')

    # Display the figure widget
    display(fig)
    return fig

def update_loss_plot(fig, train_loss, eval_loss, from_epoch=0):
    if from_epoch != 0:
        with fig.batch_update():
            fig.data[0].x = list(range(from_epoch, len(train_loss)))
            fig.data[0].y = train_loss[from_epoch:]
            fig.data[1].x = list(range(from_epoch, len(eval_loss)))
            fig.data[1].y = eval_loss[from_epoch:]
    elif len(train_loss) < 30:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss)))
            fig.data[0].y = train_loss
            fig.data[1].x = list(range(len(eval_loss)))
            fig.data[1].y = eval_loss
    else:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss) - 30, len(train_loss)))
            fig.data[0].y = train_loss[-30:]
            fig.data[1].x = list(range(len(eval_loss) - 30, len(eval_loss)))
            fig.data[1].y = eval_loss[-30:]        
    

import numpy as np
def gaussian_probability(x, y, z):
    return (1 / (np.sqrt(2 * np.pi) * z)) * np.exp(-((x - y) ** 2) / (2 * z ** 2))

from tqdm.auto import tqdm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

def do_epoch(loader, model, optimizer, device, n_epochs: int, current_epoch: int, train: bool = False,):
    if train:
        model.train()
    else:
        model.eval()
    epoch_loss = 0
    metric = R2Score(device=device)
    for inputvals, means, stds, labels in tqdm(loader, desc=f"{'Epoch' if train else 'Eval Epoch'} {current_epoch+1}/{n_epochs}"):
        inputs = torch.stack((inputvals, means, stds), dim=1).float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        metric.update(outputs.squeeze(), labels)
        if train:
            optimizer.zero_grad()
            loss = F.mse_loss(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
        else:
            loss = F.mse_loss(outputs.squeeze(), labels.float())
        epoch_loss += loss.item()
    return epoch_loss, metric.compute()


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
ann.train()

N_EPOCHS = 1000
LR = 0.01
SAVE_EVERY = 50

# Create subfolder for this loop
import os
import shutil

if os.path.exists(f"./results/{TRAINING_PREFIX}"):
    shutil.rmtree(f"./results/{TRAINING_PREFIX}")
os.makedirs(f"./results/{TRAINING_PREFIX}")

#optim = torch.optim.Adam(ann.parameters(), lr=LR)
optim = torch.optim.SGD(ann.parameters(), lr=LR)

last_loss = 0
last_eval_loss = 0

epoch_loss_list = []
eval_loss_list = [] 

fig = create_loss_plot()

for epoch in range(N_EPOCHS):
    if (epoch + 1) % 30 == 0:
        clear_output(wait=True)
        fig = create_loss_plot()
        update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    
    epoch_loss, train_R2_score = do_epoch(train_loader, ann, optim, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=True)
    eval_epoch_loss, eval_R2_score = do_epoch(test_loader, ann, None, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=False)
    eval_loss_list.append(eval_epoch_loss/len(test_loader))
    print(f"Epoch {epoch+1}, Train Loss: {epoch_loss/len(train_loader)}, Diff: {epoch_loss/len(train_loader) - last_loss}, Eval Loss: {eval_epoch_loss/len(test_loader)}, Diff Eval: {eval_epoch_loss/len(test_loader) - last_eval_loss}, Train R2 Score: {train_R2_score}, Eval R2 Score: {eval_R2_score}")
    last_loss = epoch_loss/len(train_loader)
    last_eval_loss = eval_epoch_loss/len(test_loader)
    epoch_loss_list.append(epoch_loss/len(train_loader))
    update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    if epoch % SAVE_EVERY == 0:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth")
    elif epoch == N_EPOCHS - 1:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth") 

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': 'ab202a82-b2c7-4250-9907-b5b8eaddd380',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': '27706776-45d2-4afd-a42d-eb61a34e87a0',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

Epoch 990/1000:   0%|          | 0/12500 [00:00<?, ?it/s]

Eval Epoch 990/1000:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 990, Train Loss: 0.015761945167085067, Diff: 0.00020990868780739955, Eval Loss: 0.007628089150059968, Diff Eval: -0.004385946914472152, Train R2 Score: 0.9693068265914917, Eval R2 Score: 0.9771743416786194


Epoch 991/1000:   0%|          | 0/12500 [00:00<?, ?it/s]

...

Eval Epoch 999/1000:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 999, Train Loss: 0.016313121266592717, Diff: 0.0005894456627743433, Eval Loss: 0.007837506301517133, Diff Eval: -0.0025367672169371493, Train R2 Score: 0.9682667851448059, Eval R2 Score: 0.9765476584434509


Epoch 1000/1000:   0%|          | 0/12500 [00:00<?, ?it/s]

Eval Epoch 1000/1000:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 1000, Train Loss: 0.016453526719111978, Diff: 0.00014040545251926134, Eval Loss: 0.009325923200689722, Diff Eval: 0.0014884168991725882, Train R2 Score: 0.9676513671875, Eval R2 Score: 0.9720938205718994


In [23]:
#torch.save(ann.state_dict(), "largerer_ann_new_data5.pth")


In [24]:
fig = create_loss_plot()
update_loss_plot(fig, epoch_loss_list, eval_loss_list, 30)

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': '1f97207a-01f0-4bbb-b7d3-bf540216f47b',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': 'd7e2ddbd-79a8-4e4f-8b1c-baa96d494762',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

In [48]:
# Import pickled feature scaler
import pickle

with open("extra_lin_around_mean_scaler_low_std_range_very_low_std_feature.pkl", "rb") as f:
    feature_scaler = pickle.load(f)


In [49]:
feature_scaler.transform([[0.5, 0.5, 0.5]])

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



array([[ 0.19258452,  0.85740802, -0.00171653]])

In [50]:
transformed = feature_scaler.transform([[0.5, 0.5, 0.5]])
transformed

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



array([[ 0.19258452,  0.85740802, -0.00171653]])

In [51]:
ann.eval()
transformed = feature_scaler.transform([[0.5, 0.5, 0.5]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())


c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(0.8039212730565533)

In [52]:
gaussian_probability(0.5, 0.5, 0.5)

np.float64(0.7978845608028654)

In [53]:
transformed = feature_scaler.transform([[0.1, 0.1, 0.1]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(4.004183165096531)

In [54]:
gaussian_probability(0.1, 0.1, 0.1)

np.float64(3.989422804014327)

In [55]:
transformed = feature_scaler.transform([[0.01, 0.01, 0.01]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(29.419972487232553)

In [56]:
gaussian_probability(0.01, 0.01, 0.01)

np.float64(39.894228040143275)

In [57]:
transformed = feature_scaler.transform([[0.9, 0.9, 0.9]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(0.44831923645571237)

In [58]:
gaussian_probability(0.9, 0.9, 0.9)

np.float64(0.44326920044603635)

In [59]:
transformed = feature_scaler.transform([[0.9, 0.7, 0.1]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(0.5900993465566204)

In [60]:
gaussian_probability(0.9, 0.7, 0.1)

np.float64(0.53990966513188)

In [61]:
transformed = feature_scaler.transform([[0.643, 0.7, 0.1]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(3.3220985685996185)

In [62]:
gaussian_probability(0.643, 0.7, 0.1)

np.float64(3.3912431320419234)

In [63]:
transformed = feature_scaler.transform([[4, 0.3, 0.2]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(-0.0060055248098771635)

In [64]:
gaussian_probability(4, 0.3, 0.2)

np.float64(9.57716245835995e-75)

In [42]:
train_dataset.mean.unique()

array([ 0.22517786, -0.39839161, -0.88339008,  0.74481909,  1.40303131,
       -0.84874733, -1.40303131,  0.08660687,  0.25982061,  0.77946184,
        0.6062481 , -0.57160535,  0.15589237,  1.22981757, -1.68017329,
        1.19517482, -1.64553055, -1.43767406,  0.67553359,  1.36838856,
        0.4676771 , -0.74481909, -1.22981757, -1.6108878 , -0.91803283,
        1.29910306, -0.29446336, -0.67553359, -0.64089084, -0.77946184,
        0.64089084, -0.36374886,  0.32910611, -0.15589237, -1.05660382,
        0.91803283,  1.50695955, -1.50695955,  0.81410459, -1.4723168 ,
       -0.22517786, -1.16053207, -0.05196412, -0.19053512,  1.26446031,
       -1.09124657, -0.01732137, -1.36838856,  1.12588932,  1.64553055,
       -1.26446031,  1.71481604, -1.02196108,  0.50231985,  0.84874733,
        0.29446336,  0.43303435,  0.01732137, -0.50231985,  0.05196412,
        1.5416023 ,  0.95267558,  1.16053207, -1.33374581, -0.5369626 ,
        0.57160535,  0.88339008,  1.33374581,  0.71017634,  0.12

In [43]:
train_dataset.std.unique()

array([ 0.64089084, -0.91803283, -1.19517482,  1.4723168 ,  0.25982061,
        1.64553055, -1.71481604,  1.50695955, -1.57624505, -1.6108878 ,
        1.68017329,  0.15589237, -1.40303131, -0.25982061,  0.22517786,
       -0.5369626 , -1.22981757, -0.81410459, -1.68017329,  0.19053512,
        0.74481909,  1.09124657,  0.32910611, -0.15589237, -0.08660687,
       -1.33374581,  0.36374886,  0.39839161, -0.22517786,  0.67553359,
        0.88339008, -0.12124962,  1.12588932, -0.6062481 , -1.16053207,
        1.43767406, -1.09124657, -0.71017634,  0.4676771 ,  0.91803283,
        1.6108878 ,  0.29446336,  0.84874733,  1.26446031, -0.4676771 ,
        1.40303131,  1.33374581, -1.29910306, -0.77946184,  0.81410459,
        0.43303435,  0.5369626 , -0.01732137,  1.29910306, -1.36838856,
        0.95267558,  1.36838856, -1.12588932, -1.02196108, -0.05196412,
       -1.43767406,  1.5416023 , -0.88339008,  0.01732137, -0.29446336,
        0.57160535, -0.64089084, -0.36374886, -0.84874733, -0.43

In [65]:
# std 1.25 and mean -3.04
transformed = feature_scaler.transform([[-2, -3.04, 1.25]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(0.23507488334068374)

In [66]:
gaussian_probability(-2, -3.04, 1.25)

np.float64(0.22578002723581)

In [73]:
transformed = feature_scaler.transform([[-3.04, -3.04, 0.001]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



np.float64(27.346752195786582)

In [68]:
gaussian_probability(-3.03, -3.04, 0.001)

np.float64(7.694598626688652e-20)